In [ ]:
import numpy as np 
import pandas as pd 
# !pip install sentence_transformers

In [ ]:
df = pd.read_csv('title_category.csv')
columns = df.columns.tolist()
columns[0] = 'Index'
df.columns = columns
df.head()

In [ ]:
df[pd.isnull(df['Index'])]

In [ ]:
for i in range(df.shape[0]):
    if pd.isnull(df.iloc[i,2]):
        df.iloc[i,2] = df.iloc[i+1,1]
df.dropna(subset=['Index'], inplace=True)
df.index = df['Index']
df.drop(['Index'], axis = 1, inplace=True)
df['Type of Video'] = df['Type of Video'].apply(lambda x: str(x).replace('__##__',','))

In [ ]:
df.head()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf_idf = TfidfVectorizer(min_df=2, max_features = None, strip_accents = 'unicode', norm='l2',
                            analyzer = 'char', token_pattern = r'\w{1,}',ngram_range=(1,5),
                            use_idf = 1, smooth_idf = 1, sublinear_tf = 1, stop_words = 'english')
features = tf_idf.fit_transform(df['Title of the video']).toarray()

In [ ]:
features.shape

In [ ]:
from sentence_transformers import SentenceTransformer
embedder = SentenceTransformer('bert-base-nli-mean-tokens')
bert_features = embedder.encode(df['Title of the video'].tolist())

In [ ]:
semantic_embedder = SentenceTransformer('bert-base-nli-stsb-mean-tokens')
semantic_bert_features = semantic_embedder.encode(df['Title of the video'].tolist())

In [ ]:
bert_features = np.array(bert_features)
semantic_bert_features = np.array(semantic_bert_features)

In [ ]:
final_features = np.hstack((features, bert_features, semantic_bert_features))
final_features.shape

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(tokenizer=lambda x: x.split(','))
y = vectorizer.fit_transform(df['Type of Video'])

In [ ]:
final_features.shape

In [ ]:
y.shape

In [ ]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import SGDClassifier
clf = MultiOutputClassifier(SGDClassifier(max_iter=4000)).fit(final_features, y.toarray())

In [ ]:
def generate_embedding(text):
    word_transform = tf_idf.transform([text]).toarray()[0]
    bert_transform = embedder.encode([text], show_progress_bar=False)[0]
    semantic_bert_transform = semantic_embedder.encode([text], show_progress_bar=False)[0]
    embedding = np.hstack((word_transform, bert_transform, semantic_bert_transform))
    return embedding

def get_terms(pred_list):
    return [w.title() for w in vectorizer.inverse_transform([pred_list])[0]]

def get_topics(text):
    text_embedding = generate_embedding(text)
    pred_list = clf.predict([text_embedding])[0]
    return get_terms(pred_list)

def increment_learn(text, topics):
    available_topics = vectorizer.get_feature_names()
    for topic in topics.split(','):
        if topic.lower() not in available_topics:
            return -1
    text_embedding = generate_embedding(text)
    topics = vectorizer.transform([topics]).toarray()[0]
    clf.partial_fit([text_embedding], [topics])

In [ ]:

example = 'Eric Weinstein: Revolutionary Ideas in Science, Math, and Society | Artificial Intelligence Podcast'
get_topics(example)

In [ ]:
title = 'Consciousness -- the final frontier | Dada Gunamuktananda | TEDxNoosa 2014'
get_topics(title)

In [ ]:
import pickle

# Save the model to a pickle file
with open('multioutput_classifier_model.pkl', 'wb') as model_file:
    pickle.dump(clf, model_file)


In [ ]:
import pickle

# Load the model from the pickle file
with open('multioutput_classifier_model.pkl', 'rb') as model_file:
    clf = pickle.load(model_file)


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# Define a custom tokenizer function
def custom_tokenizer(text):
    return text.split(',')

# Create the CountVectorizer with the custom tokenizer
vectorizer = CountVectorizer(tokenizer=custom_tokenizer)

# Fit and transform the data
y = vectorizer.fit_transform(df['Type of Video'])

# Save the model
import pickle
pickle.dump(clf, open('model.pkl', 'wb'))
pickle.dump(vectorizer, open('vectorizer.pkl', 'wb'))

In [ ]:
import pickle

# Load model
model = pickle.load(open('model.pkl','rb'))
vectorizer = pickle.load(open('vectorizer.pkl','rb'))

# Use model
example = 'Eric Weinstein: Revolutionary Ideas in Science, Math, and Society | Artificial Intelligence Podcast'
get_topics(example)

In [ ]:

title = 'Machine Learning for Humans, Part 1: Introduction'
get_topics(title)

In [ ]:
# import numpy as np
# from flask import Flask, request, jsonify
# import pickle

# app = Flask(__name__)
# model = pickle.load(open('model.pkl', 'rb'))
# vectorizer = pickle.load(open('vectorizer.pkl', 'rb'))

# def get_topics(text):
#     # Your model prediction logic here
#     # You can use 'model' and 'vectorizer' to make predictions
#     # Replace this with your actual prediction code
#     prediction = model.predict(vectorizer.transform([text]))
#     return prediction

# @app.route('/predict', methods=['POST'])
# def predict():
#     # For rendering results on HTML GUI
#     text = request.form['text']
#     topics = get_topics(text)
#     return render_template('index.html', prediction_text='Topics: {}'.format(topics))

# @app.route('/predict_api', methods=['POST'])
# def predict_api():
#     # For direct API calls through requests
#     data = request.get_json()
#     text = data['text']
#     topics = get_topics(text)
#     return jsonify({"topics": topics})

# if __name__ == "__main__":
#     app.run(debug=True)


In [24]:
import numpy as np 
import pandas as pd 
# !pip install sentence_transformers

df = pd.read_csv('title_category.csv')
columns = df.columns.tolist()
columns[0] = 'Index'
df.columns = columns

df[pd.isnull(df['Index'])]

for i in range(df.shape[0]):
    if pd.isnull(df.iloc[i,2]):
        df.iloc[i,2] = df.iloc[i+1,1]
df.dropna(subset=['Index'], inplace=True)
df.index = df['Index']
df.drop(['Index'], axis = 1, inplace=True)
df['Type of Video'] = df['Type of Video'].apply(lambda x: str(x).replace('__##__',','))

from sklearn.feature_extraction.text import TfidfVectorizer
tf_idf = TfidfVectorizer(min_df=2, max_features = None, strip_accents = 'unicode', norm='l2',
                            analyzer = 'char', token_pattern = r'\w{1,}',ngram_range=(1,5),
                            use_idf = 1, smooth_idf = 1, sublinear_tf = 1, stop_words = 'english')
features = tf_idf.fit_transform(df['Title of the video']).toarray()


from sentence_transformers import SentenceTransformer
embedder = SentenceTransformer('bert-base-nli-mean-tokens')
bert_features = embedder.encode(df['Title of the video'].tolist())


semantic_embedder = SentenceTransformer('bert-base-nli-stsb-mean-tokens')
semantic_bert_features = semantic_embedder.encode(df['Title of the video'].tolist())


bert_features = np.array(bert_features)
semantic_bert_features = np.array(semantic_bert_features)

final_features = np.hstack((features, bert_features, semantic_bert_features))
final_features.shape

from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(tokenizer=lambda x: x.split(','))
y = vectorizer.fit_transform(df['Type of Video'])


from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import SGDClassifier
clf = MultiOutputClassifier(SGDClassifier(max_iter=4000)).fit(final_features, y.toarray())



def generate_embedding(text):
    word_transform = tf_idf.transform([text]).toarray()[0]
    bert_transform = embedder.encode([text], show_progress_bar=False)[0]
    semantic_bert_transform = semantic_embedder.encode([text], show_progress_bar=False)[0]
    embedding = np.hstack((word_transform, bert_transform, semantic_bert_transform))
    return embedding

def get_terms(pred_list):
    return [w.title() for w in vectorizer.inverse_transform([pred_list])[0]]

def get_topics(text):
    text_embedding = generate_embedding(text)
    pred_list = clf.predict([text_embedding])[0]
    return get_terms(pred_list)

def increment_learn(text, topics):
    available_topics = vectorizer.get_feature_names()
    for topic in topics.split(','):
        if topic.lower() not in available_topics:
            return -1
    text_embedding = generate_embedding(text)
    topics = vectorizer.transform([topics]).toarray()[0]
    clf.partial_fit([text_embedding], [topics])


example = 'Eric Weinstein: Revolutionary Ideas in Science, Math, and Society | Artificial Intelligence Podcast'
get_topics(example)



/mnt/7866F86B66F82B8E/git/task-manager/Depresio/myenv/lib/python3.11/site-packages/sklearn/utils/_param_validation.py:594: FutureWarning: Passing an int for a boolean parameter is deprecated in version 1.2 and won't be supported anymore in version 1.4.
  warnings.warn(
/mnt/7866F86B66F82B8E/git/task-manager/Depresio/myenv/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:547: UserWarning: The parameter 'stop_words' will not be used since 'analyzer' != 'word'
  warnings.warn(
/mnt/7866F86B66F82B8E/git/task-manager/Depresio/myenv/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:555: UserWarning: The parameter 'token_pattern' will not be used since 'analyzer' != 'word'
  warnings.warn(
/mnt/7866F86B66F82B8E/git/task-manager/Depresio/myenv/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[]

In [27]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sentence_transformers import SentenceTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import SGDClassifier

# Read the CSV file
df = pd.read_csv('title_category.csv')

# Rename the 'Index' column
df = df.rename(columns={'Unnamed: 0': 'Index'})

# Drop rows with NaN values in the 'Index' column
df = df.dropna(subset=['Index'])

# Replace '__##__' in 'Type of Video' with commas
df['Type of Video'] = df['Type of Video'].str.replace('__##__', ',')

# Create a TF-IDF vectorizer
tf_idf = TfidfVectorizer(min_df=2, max_features=None, strip_accents='unicode',
                        norm='l2', analyzer='char', token_pattern=r'\w{1,}', ngram_range=(1, 5),
                        use_idf=1, smooth_idf=1, stop_words='english')

# Fit and transform the 'Title of the video' to TF-IDF features
features = tf_idf.fit_transform(df['Title of the video']).toarray()

# Initialize BERT-based sentence transformers
embedder = SentenceTransformer('bert-base-nli-mean-tokens')
semantic_embedder = SentenceTransformer('bert-base-nli-stsb-mean-tokens')

# Encode 'Title of the video' using BERT models
bert_features = embedder.encode(df['Title of the video'].tolist())
semantic_bert_features = semantic_embedder.encode(df['Title of the video'].tolist())

# Combine the features
final_features = np.hstack((features, bert_features, semantic_bert_features))

# Replace NaN values in 'Type of Video' with an empty string
df['Type of Video'].fillna('', inplace=True)

# Create a CountVectorizer for 'Type of Video'
vectorizer = CountVectorizer(tokenizer=lambda x: x.split(','))

# Transform 'Type of Video' into binary vectors
y = vectorizer.fit_transform(df['Type of Video'])

# Create a MultiOutputClassifier with an SGDClassifier
clf = MultiOutputClassifier(SGDClassifier(max_iter=4000)).fit(final_features, y.toarray())

def generate_embedding(text):
    word_transform = tf_idf.transform([text]).toarray()[0]
    bert_transform = embedder.encode([text], show_progress_bar=False)[0]
    semantic_bert_transform = semantic_embedder.encode([text], show_progress_bar=False)[0]
    embedding = np.hstack((word_transform, bert_transform, semantic_bert_transform))
    return embedding

def get_terms(pred_list):
    return [w.title() for w in vectorizer.inverse_transform([pred_list])[0]]

def get_topics(text):
    text_embedding = generate_embedding(text)
    pred_list = clf.predict([text_embedding])[0]
    return get_terms(pred_list)

def increment_learn(text, topics):
    available_topics = vectorizer.get_feature_names()
    for topic in topics.split(','):
        if topic.lower() not in available_topics:
            return -1
    text_embedding = generate_embedding(text)
    topics = vectorizer.transform([topics]).toarray()[0]
    clf.partial_fit([text_embedding], [topics])

example = 'Eric Weinstein: Revolutionary Ideas in Science, Math, and Society | Artificial Intelligence Podcast'
get_topics(example)


/mnt/7866F86B66F82B8E/git/task-manager/Depresio/myenv/lib/python3.11/site-packages/sklearn/utils/_param_validation.py:594: FutureWarning: Passing an int for a boolean parameter is deprecated in version 1.2 and won't be supported anymore in version 1.4.
  warnings.warn(
/mnt/7866F86B66F82B8E/git/task-manager/Depresio/myenv/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:547: UserWarning: The parameter 'stop_words' will not be used since 'analyzer' != 'word'
  warnings.warn(
/mnt/7866F86B66F82B8E/git/task-manager/Depresio/myenv/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:555: UserWarning: The parameter 'token_pattern' will not be used since 'analyzer' != 'word'
  warnings.warn(
/mnt/7866F86B66F82B8E/git/task-manager/Depresio/myenv/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


['Mathematics']